# Clone repo

In [1]:
import getpass
import subprocess
from urllib.parse import quote

username = input("GitHub username: ")
token = getpass.getpass("GitHub token: ")
repo_url = "https://github.com/ISE-Lab-AI4LIFE/SANNER_2025.git"

auth_url = repo_url.replace("https://", f"https://{quote(username)}:{quote(token)}@")

try:
    subprocess.run(["git", "clone", auth_url], check=True)
    print("✅ Repo cloned successfully!")
except subprocess.CalledProcessError as e:
    print("❌ Clone failed. Check error message below:")
    print(e.stderr)

GitHub username: hieunguyen-cyber
GitHub token: ··········
✅ Repo cloned successfully!


# Merge Hotflip result and dvide into pools

In [2]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("/content/SANNER_2025/data")  # <-- sửa ở đây
HOTFLIP_FILE = DATA_DIR / "hotflip_result" / "merged_hotflip_results.csv"
POOL_FILE = DATA_DIR / "pool.csv"

# --- Đọc dữ liệu ---
hotflip_df = pd.read_csv(HOTFLIP_FILE)
pool_df = pd.read_csv(POOL_FILE)

# --- Lấy danh sách id ---
hotflip_ids = set(hotflip_df["document_id"])
pool_ids = set(pool_df["document_id"])

# --- Phân loại ---
poisoned_doc = hotflip_df.copy()
targeted_doc = pool_df[pool_df["document_id"].isin(hotflip_ids)]
clean_doc = pool_df[~pool_df["document_id"].isin(hotflip_ids)]

# --- In thống kê ---
print(f"Tổng số document trong pool: {len(pool_df)}")
print(f"Số poisoned_doc: {len(poisoned_doc)}")
print(f"Số targeted_doc: {len(targeted_doc)}")
print(f"Số clean_doc: {len(clean_doc)}")

# --- Lưu ra file ---
OUTPUT_DIR = DATA_DIR / "hotflip_result" / "split_docs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

poisoned_doc.to_csv(OUTPUT_DIR / "poisoned_doc.csv", index=False)
targeted_doc.to_csv(OUTPUT_DIR / "targeted_doc.csv", index=False)
clean_doc.to_csv(OUTPUT_DIR / "clean_doc.csv", index=False)

print(f"✅ Đã lưu 3 file vào: {OUTPUT_DIR}")

Tổng số document trong pool: 5446
Số poisoned_doc: 157
Số targeted_doc: 157
Số clean_doc: 5289
✅ Đã lưu 3 file vào: /content/SANNER_2025/data/hotflip_result/split_docs


# Divide Poison Pool into different ratios

In [3]:
import pandas as pd
from pathlib import Path
import numpy as np


DATA_DIR = Path("/content/SANNER_2025/data")
HOTFLIP_DIR = DATA_DIR / "hotflip_result"
SPLIT_DIR = HOTFLIP_DIR / "split_docs"
POISONED_POOL_DIR = HOTFLIP_DIR / "poisoned_pool"
BASE_POOL_DIR = HOTFLIP_DIR / "base_pool"

POISONED_POOL_DIR.mkdir(parents=True, exist_ok=True)
BASE_POOL_DIR.mkdir(parents=True, exist_ok=True)

# --- Đọc 3 file ---
poisoned_doc = pd.read_csv(SPLIT_DIR / "poisoned_doc.csv")
targeted_doc = pd.read_csv(SPLIT_DIR / "targeted_doc.csv")
clean_doc = pd.read_csv(SPLIT_DIR / "clean_doc.csv")

# --- Tạo pool với nhiều ratio ---
ratios = np.array([0.001, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03])

for ratio in ratios:
    # --- Số lượng clean cần chọn ---
    n_poison = len(poisoned_doc)
    n_target = len(targeted_doc)

    n_clean_poisoned_pool = int(n_poison * (1 - ratio) / ratio)
    n_clean_base_pool = int(n_target * (1 - ratio) / ratio)

    # --- Giới hạn nếu clean_doc không đủ ---
    n_clean_poisoned_pool = min(n_clean_poisoned_pool, len(clean_doc))
    n_clean_base_pool = min(n_clean_base_pool, len(clean_doc))

    # --- Sample clean_doc ---
    clean_sample_for_poisoned = clean_doc.sample(
        n=n_clean_poisoned_pool,
        random_state=42,
        replace=False
    )
    clean_sample_for_base = clean_doc.sample(
        n=n_clean_base_pool,
        random_state=42,
        replace=False
    )

    # --- Gắn nhãn chosen ---
    poisoned_part = poisoned_doc[["document_id", "final_poisoned_doc"]].rename(
        columns={"final_poisoned_doc": "document"}
    )
    poisoned_part["chosen"] = 1

    targeted_part = targeted_doc[["document_id", "document"]].copy()
    targeted_part["chosen"] = 1

    clean_part_for_poisoned = clean_sample_for_poisoned[["document_id", "document"]].copy()
    clean_part_for_poisoned["chosen"] = 0

    clean_part_for_base = clean_sample_for_base[["document_id", "document"]].copy()
    clean_part_for_base["chosen"] = 0

    # --- Gộp lại ---
    poisoned_pool = pd.concat([poisoned_part, clean_part_for_poisoned], ignore_index=True)
    base_pool = pd.concat([targeted_part, clean_part_for_base], ignore_index=True)

    # --- Lưu ra CSV ---
    r_str = f"{ratio*100:.1f}".replace('.', '_')  # ví dụ 0_1%, 0_2%, ...
    poisoned_pool.to_csv(POISONED_POOL_DIR / f"ratio_{r_str}_percent.csv", index=False)
    base_pool.to_csv(BASE_POOL_DIR / f"ratio_{r_str}_percent.csv", index=False)

    print(f"✅ ratio={ratio*100:.1f}%: "
          f"{len(poisoned_pool)} poisoned_pool ({len(poisoned_part)} poison, {len(clean_part_for_poisoned)} clean) | "
          f"{len(base_pool)} base_pool ({len(targeted_part)} target, {len(clean_part_for_base)} clean)")

print("\n🎯 Hoàn tất tạo pool cho tất cả các ratio (0.1% → 3%)")

✅ ratio=0.1%: 5446 poisoned_pool (157 poison, 5289 clean) | 5446 base_pool (157 target, 5289 clean)
✅ ratio=0.5%: 5446 poisoned_pool (157 poison, 5289 clean) | 5446 base_pool (157 target, 5289 clean)
✅ ratio=1.0%: 5446 poisoned_pool (157 poison, 5289 clean) | 5446 base_pool (157 target, 5289 clean)
✅ ratio=1.5%: 5446 poisoned_pool (157 poison, 5289 clean) | 5446 base_pool (157 target, 5289 clean)
✅ ratio=2.0%: 5446 poisoned_pool (157 poison, 5289 clean) | 5446 base_pool (157 target, 5289 clean)
✅ ratio=2.5%: 5446 poisoned_pool (157 poison, 5289 clean) | 5446 base_pool (157 target, 5289 clean)
✅ ratio=3.0%: 5233 poisoned_pool (157 poison, 5076 clean) | 5233 base_pool (157 target, 5076 clean)

🎯 Hoàn tất tạo pool cho tất cả các ratio (0.1% → 3%)


# Divide Targetted Pool into different ratios

In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np

# --- Cấu hình thư mục ---
DATA_DIR = Path("/content/SANNER_2025/data")
HOTFLIP_DIR = DATA_DIR / "hotflip_result"
SPLIT_DIR = HOTFLIP_DIR / "split_docs"
TARGETTED_POOL_DIR = HOTFLIP_DIR / "targetted_pool"
BASE_POOL_DIR = HOTFLIP_DIR / "base_pool_targetted"

TARGETTED_POOL_DIR.mkdir(parents=True, exist_ok=True)
BASE_POOL_DIR.mkdir(parents=True, exist_ok=True)

# --- Đọc dữ liệu ---
poisoned_doc = pd.read_csv(SPLIT_DIR / "poisoned_doc.csv")
targeted_doc = pd.read_csv(SPLIT_DIR / "targeted_doc.csv")
clean_doc = pd.read_csv(SPLIT_DIR / "clean_doc.csv")

# --- Các tỷ lệ cần tạo ---
ratios = np.array([0.001, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03])

for ratio in ratios:
    # --- Số lượng doc ---
    n_target = len(targeted_doc)
    n_poison = len(poisoned_doc)

    n_clean_targetted_pool = int(n_target * (1 - ratio) / ratio)
    n_clean_base_pool = int(n_poison * (1 - ratio) / ratio)

    # --- Giới hạn nếu clean_doc không đủ ---
    n_clean_targetted_pool = min(n_clean_targetted_pool, len(clean_doc))
    n_clean_base_pool = min(n_clean_base_pool, len(clean_doc))

    # --- Sample clean doc ---
    clean_sample_for_targetted = clean_doc.sample(
        n=n_clean_targetted_pool,
        random_state=42,
        replace=False
    )
    clean_sample_for_base = clean_doc.sample(
        n=n_clean_base_pool,
        random_state=42,
        replace=False
    )

    # --- Chuẩn bị phần targeted ---
    targeted_part = targeted_doc[["document_id", "document"]].copy()
    targeted_part["chosen"] = 1

    poisoned_part = poisoned_doc[["document_id", "final_poisoned_doc"]].rename(
        columns={"final_poisoned_doc": "document"}
    )
    poisoned_part["chosen"] = 1

    clean_part_for_targetted = clean_sample_for_targetted[["document_id", "document"]].copy()
    clean_part_for_targetted["chosen"] = 0

    clean_part_for_base = clean_sample_for_base[["document_id", "document"]].copy()
    clean_part_for_base["chosen"] = 0

    # --- Gộp lại ---
    targetted_pool = pd.concat([targeted_part, clean_part_for_targetted], ignore_index=True)
    base_pool = pd.concat([poisoned_part, clean_part_for_base], ignore_index=True)

    # --- Lưu file ---
    r_str = f"{ratio*100:.1f}".replace('.', '_')
    targetted_pool.to_csv(TARGETTED_POOL_DIR / f"ratio_{r_str}_percent.csv", index=False)
    base_pool.to_csv(BASE_POOL_DIR / f"ratio_{r_str}_percent.csv", index=False)

    print(f"🎯 ratio={ratio*100:.1f}%: "
          f"{len(targetted_pool)} targetted_pool ({len(targeted_part)} target, {len(clean_part_for_targetted)} clean) | "
          f"{len(base_pool)} base_pool ({len(poisoned_part)} poison, {len(clean_part_for_base)} clean)")

print("\n✅ Hoàn tất tạo pool targetted cho tất cả các ratio (0.1% → 3%)")

# Embedding Calculation

In [7]:
import pandas as pd
from pathlib import Path
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
from tqdm import tqdm

DATA_DIR = Path("/content/SANNER_2025/data")
HOTFLIP_DIR = DATA_DIR / "hotflip_result"
SPLIT_DIR = HOTFLIP_DIR / "split_docs"
EMB_DIR = HOTFLIP_DIR / "embeddings"
EMB_DIR.mkdir(parents=True, exist_ok=True)

# --- Thiết bị ---
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device=device)

# --- Sliding window config ---
MAX_TOKENS = 256
STRIDE = 128
BATCH_SIZE = 32

def sliding_window_embed(text, model, max_len=MAX_TOKENS, stride=STRIDE):
    """Cắt văn bản dài và trung bình embedding."""
    tokens = model.tokenizer.tokenize(text)
    if len(tokens) <= max_len:
        return model.encode([text], convert_to_tensor=True)
    else:
        chunks = []
        for i in range(0, len(tokens), stride):
            chunk = tokens[i:i + max_len]
            chunk_text = model.tokenizer.convert_tokens_to_string(chunk)
            chunks.append(chunk_text)
            if i + max_len >= len(tokens):
                break
        embeddings = model.encode(chunks, convert_to_tensor=True, batch_size=BATCH_SIZE)
        return embeddings.mean(dim=0, keepdim=True)

# --- Danh sách file và cột tương ứng ---
file_configs = {
    "poisoned_doc": "final_poisoned_doc",
    "targeted_doc": "document",
    "clean_doc": "document"
}

for name, text_col in file_configs.items():
    path = SPLIT_DIR / f"{name}.csv"
    df = pd.read_csv(path)

    print(f"\n🔹 Encoding {name} ({len(df)} docs) using column '{text_col}'...")
    all_embs = []
    for text in tqdm(df[text_col], desc=f"Encoding {name}"):
        emb = sliding_window_embed(str(text), model)
        all_embs.append(emb.cpu().numpy())

    all_embs = np.vstack(all_embs)
    np.save(EMB_DIR / f"{name}_emb.npy", all_embs)

    # chỉ giữ lại các cột liên quan để truy map theo id
    meta_cols = ["document_id", text_col]
    df[meta_cols].to_csv(EMB_DIR / f"{name}_meta.csv", index=False)

    print(f"✅ Done: {name} | shape = {all_embs.shape}")

print("\n🎯 Hoàn tất tính và lưu embedding cho 3 file gốc (clean / targeted / poisoned)")


🔹 Encoding poisoned_doc (157 docs) using column 'final_poisoned_doc'...


Encoding poisoned_doc: 100%|██████████| 157/157 [00:11<00:00, 13.40it/s]


✅ Done: poisoned_doc | shape = (157, 768)

🔹 Encoding targeted_doc (157 docs) using column 'document'...


Encoding targeted_doc: 100%|██████████| 157/157 [00:07<00:00, 21.95it/s]


✅ Done: targeted_doc | shape = (157, 768)

🔹 Encoding clean_doc (5289 docs) using column 'document'...


Encoding clean_doc: 100%|██████████| 5289/5289 [05:18<00:00, 16.60it/s]


✅ Done: clean_doc | shape = (5289, 768)

🎯 Hoàn tất tính và lưu embedding cho 3 file gốc (clean / targeted / poisoned)


In [9]:
# --- Encode queries (thêm vào sau khi đã encode 3 file gốc) ---
TEST_FILE = DATA_DIR / "test.csv"  # đường dẫn tới file queries

if TEST_FILE.exists():
    queries_df = pd.read_csv(TEST_FILE)
    # đảm bảo cột tên đúng
    assert "queries" in queries_df.columns or "query" in queries_df.columns, \
        "File test.csv cần có cột 'queries' (hoặc 'query')"

    # chọn tên cột đúng nếu khác
    q_col = "queries" if "queries" in queries_df.columns else "query"

    print(f"\n🔹 Encoding queries ({len(queries_df)} queries) using column '{q_col}'...")
    query_embs = []
    for text in tqdm(queries_df[q_col].fillna("").astype(str), desc="Encoding queries"):
        emb = sliding_window_embed(text, model)
        query_embs.append(emb.cpu().numpy())

    query_embs = np.vstack(query_embs)  # (n_queries, dim)
    np.save(EMB_DIR / "queries_emb.npy", query_embs)
    queries_df[["queries_id", q_col]] if "queries_id" in queries_df.columns else queries_df[[q_col]]
    # lưu metadata (giữ queries_id nếu có)
    meta_cols = ["queries_id", q_col] if "queries_id" in queries_df.columns else [q_col]
    queries_df[meta_cols].to_csv(EMB_DIR / "queries_meta.csv", index=False)

    print(f"✅ Done: queries embeddings saved with shape = {query_embs.shape}")
else:
    print(f"⚠️ Không tìm thấy file queries ở: {TEST_FILE}. Bỏ qua bước encode queries.")


🔹 Encoding queries (15232 queries) using column 'queries'...


Encoding queries: 100%|██████████| 15232/15232 [05:59<00:00, 42.39it/s]


✅ Done: queries embeddings saved with shape = (15232, 768)


# 1st test
## 📊 Explanation of Metrics

- **ratio** → Poison ratio of the document pool (e.g., 10%, 30%, 50%)
- **k** → Top-K documents considered for each query
- **custom_precision@k** → % of poisoned documents appearing in top-K results  
  → Higher = stronger poisoning effect
- **num_queries** → Number of evaluated query embeddings
- **num_poisoned_docs** → Number of poisoned (inserted) documents in the pool
- **num_pool_docs** → Total number of documents in the retrieval pool
## Poison Doc

In [ ]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

# ====================== CẤU HÌNH ======================

DATA_DIR = Path("/content/SANNER_2025/data")
HOTFLIP_DIR = DATA_DIR / "hotflip_result"
POISONED_POOL_DIR = HOTFLIP_DIR / "poisoned_pool"
EMB_DIR = HOTFLIP_DIR / "embeddings"
TEST_FILE = DATA_DIR / "test.csv"

# chọn thiết bị
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

TOP_K_LIST = [1, 3, 5, 10]

# ====================== HÀM CHÍNH ======================
def custom_precision_at_k_poison(query_embeds, pool_embs, pool_ids, poison_ids, K=10):
    sim = torch.matmul(query_embeds, pool_embs.T)
    topk_idx = torch.topk(sim, k=K, dim=1).indices

    total_hits = 0
    for i in range(sim.shape[0]):
        topk_docs = [pool_ids[idx] for idx in topk_idx[i].tolist()]
        total_hits += sum(d in poison_ids for d in topk_docs)

    N = query_embeds.shape[0]
    # ✅ scaled để kết quả nằm trong khoảng 1–10
    return (100 * total_hits) / (K * N)

# ====================== TẢI EMBEDDING ======================
print("🔹 Loading embeddings ...")

# queries
queries_emb = torch.tensor(np.load(EMB_DIR / "queries_emb.npy"), dtype=torch.float32, device=device)
queries_meta = pd.read_csv(EMB_DIR / "queries_meta.csv")

# poisoned / targeted / clean
meta_poisoned = pd.read_csv(EMB_DIR / "poisoned_doc_meta.csv")
meta_targeted = pd.read_csv(EMB_DIR / "targeted_doc_meta.csv")
meta_clean = pd.read_csv(EMB_DIR / "clean_doc_meta.csv")

emb_poisoned = torch.tensor(np.load(EMB_DIR / "poisoned_doc_emb.npy"), dtype=torch.float32, device=device)
emb_targeted = torch.tensor(np.load(EMB_DIR / "targeted_doc_emb.npy"), dtype=torch.float32, device=device)
emb_clean = torch.tensor(np.load(EMB_DIR / "clean_doc_emb.npy"), dtype=torch.float32, device=device)

# Map ID → embedding tensor
embedding_map = {
    **{doc_id: emb_poisoned[i] for i, doc_id in enumerate(meta_poisoned["document_id"])},
    **{doc_id: emb_targeted[i] for i, doc_id in enumerate(meta_targeted["document_id"])},
    **{doc_id: emb_clean[i] for i, doc_id in enumerate(meta_clean["document_id"])},
}

# ====================== CHẠY CHO TỪNG RATIO ======================
results = []

ratio_files = sorted(POISONED_POOL_DIR.glob("ratio_*percent.csv"))

for f in tqdm(ratio_files, desc="Evaluating poisoned_pool ratios"):
    ratio_name = f.stem
    df = pd.read_csv(f)

    # document_id → embedding
    valid_ids = [doc_id for doc_id in df["document_id"] if doc_id in embedding_map]
    pool_embs = torch.stack([embedding_map[doc_id] for doc_id in valid_ids]).to(device)
    pool_ids = list(valid_ids)

    # danh sách poisoned docs
    poison_ids = set(df.loc[df["chosen"] == 1, "document_id"].tolist())

    # tính cho từng K
    for K in TOP_K_LIST:
        p_at_k = custom_precision_at_k_poison(
            queries_emb, pool_embs, pool_ids, poison_ids, K=K
        )
        results.append({
            "ratio": ratio_name,
            "k": K,
            "custom_precision@k": round(float(p_at_k), 6),   # ✅ bỏ .item(), ép về float an toàn
            "num_queries": queries_emb.shape[0],
            "num_poisoned_docs": len(poison_ids),
            "num_pool_docs": len(pool_ids),
        })

# ====================== LƯU KẾT QUẢ ======================
results_df = pd.DataFrame(results)
out_path = HOTFLIP_DIR / "result" / "custom_p_at_k_poison.csv"
results_df.to_csv(out_path, index=False)

print(f"\n✅ Saved results to {out_path}")
print(results_df.head(10))

🔹 Loading embeddings ...


Evaluating poisoned_pool ratios: 100%|██████████| 7/7 [00:11<00:00,  1.67s/it]


✅ Saved results to /content/SANNER_2025/data/hotflip_result/custom_p_at_k_poison.csv
               ratio   k  custom_precision@k  num_queries  num_poisoned_docs  \
0  ratio_0_1_percent   1            3.571429        15232                157   
1  ratio_0_1_percent   3            3.437938        15232                157   
2  ratio_0_1_percent   5            3.453256        15232                157   
3  ratio_0_1_percent  10            3.409270        15232                157   
4  ratio_0_5_percent   1            3.571429        15232                157   
5  ratio_0_5_percent   3            3.437938        15232                157   
6  ratio_0_5_percent   5            3.453256        15232                157   
7  ratio_0_5_percent  10            3.409270        15232                157   
8  ratio_1_0_percent   1            3.571429        15232                157   
9  ratio_1_0_percent   3            3.437938        15232                157   

   num_pool_docs  
0           54

## Targetted Doc

In [ ]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

# ====================== CẤU HÌNH ======================

DATA_DIR = Path("/content/SANNER_2025/data")
HOTFLIP_DIR = DATA_DIR / "hotflip_result"
TARGETTED_POOL_DIR = HOTFLIP_DIR / "targetted_pool"
EMB_DIR = HOTFLIP_DIR / "embeddings"
TEST_FILE = DATA_DIR / "test.csv"

# chọn thiết bị
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

TOP_K_LIST = [1, 3, 5, 10]

# ====================== HÀM CHÍNH ======================
def custom_precision_at_k_target(query_embeds, pool_embs, pool_ids, target_ids, K=10):
    """
    Tính custom precision@k cho targeted attack:
    - query_embeds: embedding của queries
    - pool_embs: embedding của các document trong targetted_pool
    - target_ids: tập document_id bị target (chosen == 1)
    """
    sim = torch.matmul(query_embeds, pool_embs.T)
    topk_idx = torch.topk(sim, k=K, dim=1).indices

    total_hits = 0
    for i in range(sim.shape[0]):
        topk_docs = [pool_ids[idx] for idx in topk_idx[i].tolist()]
        total_hits += sum(d in target_ids for d in topk_docs)

    N = query_embeds.shape[0]
    # ✅ scale để kết quả có thể so sánh được, nằm trong khoảng 1–10
    return (100 * total_hits) / (K * N)

# ====================== TẢI EMBEDDING ======================
print("🔹 Loading embeddings ...")

# queries
queries_emb = torch.tensor(np.load(EMB_DIR / "queries_emb.npy"), dtype=torch.float32, device=device)
queries_meta = pd.read_csv(EMB_DIR / "queries_meta.csv")

# poisoned / targeted / clean
meta_poisoned = pd.read_csv(EMB_DIR / "poisoned_doc_meta.csv")
meta_targeted = pd.read_csv(EMB_DIR / "targeted_doc_meta.csv")
meta_clean = pd.read_csv(EMB_DIR / "clean_doc_meta.csv")

emb_poisoned = torch.tensor(np.load(EMB_DIR / "poisoned_doc_emb.npy"), dtype=torch.float32, device=device)
emb_targeted = torch.tensor(np.load(EMB_DIR / "targeted_doc_emb.npy"), dtype=torch.float32, device=device)
emb_clean = torch.tensor(np.load(EMB_DIR / "clean_doc_emb.npy"), dtype=torch.float32, device=device)

# Map ID → embedding tensor
embedding_map = {
    **{doc_id: emb_poisoned[i] for i, doc_id in enumerate(meta_poisoned["document_id"])},
    **{doc_id: emb_targeted[i] for i, doc_id in enumerate(meta_targeted["document_id"])},
    **{doc_id: emb_clean[i] for i, doc_id in enumerate(meta_clean["document_id"])},
}

# ====================== CHẠY CHO TỪNG RATIO ======================
results = []

ratio_files = sorted(TARGETTED_POOL_DIR.glob("ratio_*percent.csv"))

for f in tqdm(ratio_files, desc="Evaluating targetted_pool ratios"):
    ratio_name = f.stem
    df = pd.read_csv(f)

    # document_id → embedding
    valid_ids = [doc_id for doc_id in df["document_id"] if doc_id in embedding_map]
    pool_embs = torch.stack([embedding_map[doc_id] for doc_id in valid_ids]).to(device)
    pool_ids = list(valid_ids)

    # danh sách targeted docs
    target_ids = set(df.loc[df["chosen"] == 1, "document_id"].tolist())

    # tính cho từng K
    for K in TOP_K_LIST:
        p_at_k = custom_precision_at_k_target(
            queries_emb, pool_embs, pool_ids, target_ids, K=K
        )
        results.append({
            "ratio": ratio_name,
            "k": K,
            "custom_precision@k": round(float(p_at_k), 6),
            "num_queries": queries_emb.shape[0],
            "num_target_docs": len(target_ids),
            "num_pool_docs": len(pool_ids),
        })

# ====================== LƯU KẾT QUẢ ======================
results_df = pd.DataFrame(results)
out_path = HOTFLIP_DIR / "result" / "custom_p_at_k_targetted.csv"
results_df.to_csv(out_path, index=False)

print(f"\n✅ Saved results to {out_path}")
print(results_df.head(10))

# 2nd test
## 📊 Explanation of Metrics

- **ratio** → Poison ratio of the document pool (e.g., 10%, 30%, 50%)
- **avg_poison_in_topK** → Average number of poisoned documents appearing in the top-K results per query  
  → Higher = stronger poisoning or retrieval corruption
- **num_queries** → Total number of query embeddings evaluated
- **num_poisoned_docs** → Number of poisoned (manipulated) documents in the pool
- **num_pool_docs** → Total number of documents available for retrieval
## Poisoned Doc

In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
from tqdm import tqdm

# --- Cấu hình ---
DATA_DIR = Path("/content/SANNER_2025/data") 
HOTFLIP_DIR = DATA_DIR / "hotflip_result"
POISONED_POOL_DIR = HOTFLIP_DIR / "poisoned_pool"
EMB_DIR = HOTFLIP_DIR / "embeddings"
TEST_FILE = DATA_DIR / "test.csv"

# --- Tham số ---
K = 10

# --- Thiết bị ---
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# --- Load queries ---
queries_emb = torch.tensor(np.load(EMB_DIR / "queries_emb.npy"), dtype=torch.float32, device=device)
queries_meta = pd.read_csv(EMB_DIR / "queries_meta.csv")

# --- Load embeddings gốc ---
meta_poisoned = pd.read_csv(EMB_DIR / "poisoned_doc_meta.csv")
meta_targeted = pd.read_csv(EMB_DIR / "targeted_doc_meta.csv")
meta_clean = pd.read_csv(EMB_DIR / "clean_doc_meta.csv")

emb_poisoned = torch.tensor(np.load(EMB_DIR / "poisoned_doc_emb.npy"), dtype=torch.float32, device=device)
emb_targeted = torch.tensor(np.load(EMB_DIR / "targeted_doc_emb.npy"), dtype=torch.float32, device=device)
emb_clean = torch.tensor(np.load(EMB_DIR / "clean_doc_emb.npy"), dtype=torch.float32, device=device)

embedding_map = {
    **{doc_id: emb_poisoned[i] for i, doc_id in enumerate(meta_poisoned["document_id"])},
    **{doc_id: emb_targeted[i] for i, doc_id in enumerate(meta_targeted["document_id"])},
    **{doc_id: emb_clean[i] for i, doc_id in enumerate(meta_clean["document_id"])},
}

# --- Tính cho từng ratio ---
ratio_files = sorted(POISONED_POOL_DIR.glob("ratio_*percent.csv"))
results = []

for f in tqdm(ratio_files, desc="Computing avg poison-in-topK"):
    ratio_name = f.stem
    df = pd.read_csv(f)

    valid_ids = [doc_id for doc_id in df["document_id"] if doc_id in embedding_map]
    pool_embs = torch.stack([embedding_map[doc_id] for doc_id in valid_ids]).to(device)
    pool_ids = list(valid_ids)
    poison_ids = set(df.loc[df["chosen"] == 1, "document_id"].tolist())

    # Cosine similarity
    sim = torch.matmul(queries_emb, pool_embs.T)
    topk_idx = torch.topk(sim, k=K, dim=1).indices

    poison_counts = []
    for i in range(sim.shape[0]):
        topk_docs = [pool_ids[idx] for idx in topk_idx[i].tolist()]
        n_poison = sum(d in poison_ids for d in topk_docs)
        poison_counts.append(n_poison)

    avg_poison_topk = float(np.mean(poison_counts))
    results.append({
        "ratio": ratio_name,
        f"avg_poison_in_top{K}": round(avg_poison_topk, 4),
        "num_queries": len(queries_meta),
        "num_poisoned_docs": len(poison_ids),
        "num_pool_docs": len(pool_ids),
    })

# --- Xuất kết quả ---
results_df = pd.DataFrame(results)
out_path = HOTFLIP_DIR / "result" / f"avg_poison_in_top{K}.csv"
results_df.to_csv(out_path, index=False)

print(f"\n✅ Saved results to {out_path}")
print(results_df.head(10))

## Targetted Doc

In [ ]:
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

# ====================== CẤU HÌNH ======================
DATA_DIR = Path("/content/SANNER_2025/data")
HOTFLIP_DIR = DATA_DIR / "hotflip_result"
TARGETTED_POOL_DIR = HOTFLIP_DIR / "targetted_pool"   # ✅ đổi sang targetted
EMB_DIR = HOTFLIP_DIR / "embeddings"
TEST_FILE = DATA_DIR / "test.csv"

# ====================== THAM SỐ ======================
K = 10  # top-K

# ====================== THIẾT BỊ ======================
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"✅ Using device: {device}")

# ====================== LOAD QUERY ======================
queries_emb = torch.tensor(np.load(EMB_DIR / "queries_emb.npy"), dtype=torch.float32, device=device)
queries_meta = pd.read_csv(EMB_DIR / "queries_meta.csv")

# ====================== LOAD EMBEDDING VÀ META ======================
def load_emb_and_meta(name):
    meta = pd.read_csv(EMB_DIR / f"{name}_doc_meta.csv")
    emb = torch.tensor(np.load(EMB_DIR / f"{name}_doc_emb.npy"), dtype=torch.float32, device=device)
    return meta, emb

meta_poisoned, emb_poisoned = load_emb_and_meta("poisoned")
meta_targeted, emb_targeted = load_emb_and_meta("targeted")
meta_clean, emb_clean = load_emb_and_meta("clean")

# Map ID → embedding
embedding_map = {
    **{doc_id: emb_poisoned[i] for i, doc_id in enumerate(meta_poisoned["document_id"])},
    **{doc_id: emb_targeted[i] for i, doc_id in enumerate(meta_targeted["document_id"])},
    **{doc_id: emb_clean[i] for i, doc_id in enumerate(meta_clean["document_id"])},
}

print(f"📦 Total embeddings loaded: {len(embedding_map):,}")

# ====================== DUYỆT CÁC FILE RATIO ======================
ratio_files = sorted(TARGETTED_POOL_DIR.glob("ratio_*percent.csv"))
if not ratio_files:
    raise FileNotFoundError(f"No ratio_*percent.csv found in {TARGETTED_POOL_DIR}")

results = []

for f in tqdm(ratio_files, desc="Computing avg targeted-in-topK"):
    ratio_name = f.stem
    df = pd.read_csv(f)

    # Lọc doc có embedding
    valid_ids = [doc_id for doc_id in df["document_id"] if doc_id in embedding_map]
    if not valid_ids:
        print(f"⚠️ Skip {ratio_name} — no valid document IDs found.")
        continue

    pool_embs = torch.stack([embedding_map[doc_id] for doc_id in valid_ids]).to(device)
    pool_ids = list(valid_ids)

    # Lấy targeted docs
    targeted_ids = set(df.loc[df["chosen"] == 1, "document_id"])

    # --- Cosine similarity ---
    sim = torch.matmul(queries_emb, pool_embs.T)
    topk_idx = torch.topk(sim, k=min(K, sim.shape[1]), dim=1).indices

    # --- Đếm số doc targeted trong top-K ---
    targeted_counts = []
    for i in range(sim.shape[0]):
        topk_docs = [pool_ids[idx] for idx in topk_idx[i].tolist()]
        n_targeted = sum(doc_id in targeted_ids for doc_id in topk_docs)
        targeted_counts.append(n_targeted)

    avg_targeted_topk = float(np.mean(targeted_counts))
    results.append({
        "ratio": ratio_name,
        f"avg_targeted_in_top{K}": round(avg_targeted_topk, 4),
        "num_queries": len(queries_meta),
        "num_targeted_docs": len(targeted_ids),
        "num_pool_docs": len(pool_ids),
    })

# ====================== XUẤT KẾT QUẢ ======================
results_df = pd.DataFrame(results)
out_path = HOTFLIP_DIR / "result" / f"avg_targeted_in_top{K}.csv"
results_df.to_csv(out_path, index=False)

print(f"\n✅ Saved results to {out_path}")
print(results_df.head(10))

# 3rd test
## 📊 Explanation of Attack Evaluation Metrics

- **ratio** → Poison ratio of the document pool (e.g., 10%, 30%, 50%)
- **k** → Top-K documents considered for retrieval evaluation
- **precision@k_raw** → Total number of poisoned docs found across all queries (not normalized)
- **precision@k_norm** → Normalized precision = hits / (K × number of queries)
- **precision@k_scaled** → Precision scaled to percentage (0–100%) for easier comparison
- **recall@k** → Fraction of all poisoned docs successfully retrieved within top-K
- **attack_success@k** → Attack Success Rate (ASR): proportion of queries that retrieved at least one poisoned doc
- **mrr@k** → Mean Reciprocal Rank — average of 1/rank of the first poisoned doc found  
  → Higher = poisoned docs appear earlier in ranking
- **avg_rank_first_poison** → Average rank position of the first poisoned doc  
  → Lower = stronger attack effect
- **num_queries** → Number of evaluated queries
- **num_poisoned_docs** → Total poisoned documents in the pool
- **num_pool_docs** → Total documents available for retrieval
## Poisoned Doc

In [ ]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

# --- Hàm tiện ích chính ---
def compute_attack_metrics(query_embeds, pool_embs, pool_ids, poison_ids, K=10):
    sim = torch.matmul(query_embeds, pool_embs.T)  # [n_q, n_docs]
    n_queries = sim.shape[0]
    n_poison = len(poison_ids)

    # Lấy top-K index
    topk_idx = torch.topk(sim, k=K, dim=1).indices

    total_hits = 0
    recall_hits = 0
    success_flags = []
    reciprocal_ranks = []
    first_ranks = []

    for i in range(n_queries):
        scores = sim[i]
        ranked_indices = torch.argsort(scores, descending=True)
        ranked_ids = [pool_ids[idx] for idx in ranked_indices.tolist()]

        # chỉ số top-K
        topk_docs = [pool_ids[idx] for idx in topk_idx[i].tolist()]
        poison_in_topk = sum(d in poison_ids for d in topk_docs)
        total_hits += poison_in_topk

        # Attack Success Rate
        success_flags.append(1 if poison_in_topk > 0 else 0)

        # Recall@K (tổng poison tìm được / tổng poison có)
        recall_hits += poison_in_topk

        # Rank-based metrics
        first_poison_rank = None
        for rank, doc in enumerate(ranked_ids, start=1):
            if doc in poison_ids:
                first_poison_rank = rank
                break

        if first_poison_rank:
            reciprocal_ranks.append(1 / first_poison_rank)
            first_ranks.append(first_poison_rank)
        else:
            reciprocal_ranks.append(0)
            first_ranks.append(np.nan)

    # Precision (scaled + normalized)
    p_raw = total_hits / K
    p_norm = total_hits / (K * n_queries)
    p_scaled = (100 * total_hits) / (K * n_queries)

    # Recall@K
    recall_at_k = recall_hits / max(n_poison, 1)

    # Attack Success Rate
    asr = np.mean(success_flags)

    # MRR & ARFP
    mrr = np.mean(reciprocal_ranks)
    avg_rank = np.nanmean(first_ranks)

    return {
        "precision@k_raw": round(p_raw, 4),
        "precision@k_norm": round(p_norm, 4),
        "precision@k_scaled": round(p_scaled, 4),
        "recall@k": round(recall_at_k, 4),
        "attack_success@k": round(asr, 4),
        "mrr@k": round(mrr, 4),
        "avg_rank_first_poison": round(avg_rank, 2),
    }

results = []

for f in tqdm(ratio_files, desc="Evaluating all metrics"):
    ratio_name = f.stem
    df = pd.read_csv(f)

    valid_ids = [doc_id for doc_id in df["document_id"] if doc_id in embedding_map]
    pool_embs = torch.stack([embedding_map[doc_id] for doc_id in valid_ids]).to(device)
    pool_ids = list(valid_ids)
    poison_ids = set(df.loc[df["chosen"] == 1, "document_id"].tolist())

    for K in [1, 3, 5, 10]:
        metrics = compute_attack_metrics(queries_emb, pool_embs, pool_ids, poison_ids, K=K)
        results.append({
            "ratio": ratio_name,
            "k": K,
            "num_queries": queries_emb.shape[0],
            "num_poisoned_docs": len(poison_ids),
            "num_pool_docs": len(pool_ids),
            **metrics
        })

# Lưu ra CSV
results_df = pd.DataFrame(results)
out_path = HOTFLIP_DIR / "result" / "attack_metrics_full.csv"
results_df.to_csv(out_path, index=False)

print(f"\n✅ Saved metrics to {out_path}")
print(results_df.head(10))

## Targetted Doc

In [ ]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

# ====================== CẤU HÌNH ======================
DATA_DIR = Path("/content/SANNER_2025/data")
HOTFLIP_DIR = DATA_DIR / "hotflip_result"
TARGETTED_POOL_DIR = HOTFLIP_DIR / "targetted_pool"
EMB_DIR = HOTFLIP_DIR / "embeddings"

# chọn thiết bị
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

# ====================== HÀM CHÍNH ======================
def compute_attack_metrics_target(query_embeds, pool_embs, pool_ids, target_ids, K=10):
    """
    Tính các chỉ số đánh giá cho targeted attack.
    - query_embeds: embedding của các query
    - pool_embs: embedding của các document trong targetted pool
    - target_ids: danh sách document_id bị tấn công (chosen == 1)
    """
    sim = torch.matmul(query_embeds, pool_embs.T)  # [n_q, n_docs]
    n_queries = sim.shape[0]
    n_target = len(target_ids)

    # Lấy top-K index
    topk_idx = torch.topk(sim, k=K, dim=1).indices

    total_hits = 0
    recall_hits = 0
    success_flags = []
    reciprocal_ranks = []
    first_ranks = []

    for i in range(n_queries):
        scores = sim[i]
        ranked_indices = torch.argsort(scores, descending=True)
        ranked_ids = [pool_ids[idx] for idx in ranked_indices.tolist()]

        # Chỉ số top-K
        topk_docs = [pool_ids[idx] for idx in topk_idx[i].tolist()]
        target_in_topk = sum(d in target_ids for d in topk_docs)
        total_hits += target_in_topk

        # Attack Success Rate
        success_flags.append(1 if target_in_topk > 0 else 0)

        # Recall@K (tổng target tìm được / tổng target có)
        recall_hits += target_in_topk

        # Rank-based metrics
        first_target_rank = None
        for rank, doc in enumerate(ranked_ids, start=1):
            if doc in target_ids:
                first_target_rank = rank
                break

        if first_target_rank:
            reciprocal_ranks.append(1 / first_target_rank)
            first_ranks.append(first_target_rank)
        else:
            reciprocal_ranks.append(0)
            first_ranks.append(np.nan)

    # Precision (scaled + normalized)
    p_raw = total_hits / K
    p_norm = total_hits / (K * n_queries)
    p_scaled = (100 * total_hits) / (K * n_queries)

    # Recall@K
    recall_at_k = recall_hits / max(n_target, 1)

    # Attack Success Rate
    asr = np.mean(success_flags)

    # MRR & Avg. Rank
    mrr = np.mean(reciprocal_ranks)
    avg_rank = np.nanmean(first_ranks)

    return {
        "precision@k_raw": round(p_raw, 4),
        "precision@k_norm": round(p_norm, 4),
        "precision@k_scaled": round(p_scaled, 4),
        "recall@k": round(recall_at_k, 4),
        "attack_success@k": round(asr, 4),
        "mrr@k": round(mrr, 4),
        "avg_rank_first_target": round(avg_rank, 2),
    }

# ====================== TẢI EMBEDDING ======================
print("🔹 Loading embeddings ...")

queries_emb = torch.tensor(np.load(EMB_DIR / "queries_emb.npy"), dtype=torch.float32, device=device)
meta_poisoned = pd.read_csv(EMB_DIR / "poisoned_doc_meta.csv")
meta_targeted = pd.read_csv(EMB_DIR / "targeted_doc_meta.csv")
meta_clean = pd.read_csv(EMB_DIR / "clean_doc_meta.csv")

emb_poisoned = torch.tensor(np.load(EMB_DIR / "poisoned_doc_emb.npy"), dtype=torch.float32, device=device)
emb_targeted = torch.tensor(np.load(EMB_DIR / "targeted_doc_emb.npy"), dtype=torch.float32, device=device)
emb_clean = torch.tensor(np.load(EMB_DIR / "clean_doc_emb.npy"), dtype=torch.float32, device=device)

embedding_map = {
    **{doc_id: emb_poisoned[i] for i, doc_id in enumerate(meta_poisoned["document_id"])},
    **{doc_id: emb_targeted[i] for i, doc_id in enumerate(meta_targeted["document_id"])},
    **{doc_id: emb_clean[i] for i, doc_id in enumerate(meta_clean["document_id"])},
}

# ====================== CHẠY CHO TỪNG RATIO ======================
results = []

ratio_files = sorted(TARGETTED_POOL_DIR.glob("ratio_*percent.csv"))

for f in tqdm(ratio_files, desc="Evaluating all metrics for targeted attack"):
    ratio_name = f.stem
    df = pd.read_csv(f)

    valid_ids = [doc_id for doc_id in df["document_id"] if doc_id in embedding_map]
    pool_embs = torch.stack([embedding_map[doc_id] for doc_id in valid_ids]).to(device)
    pool_ids = list(valid_ids)
    target_ids = set(df.loc[df["chosen"] == 1, "document_id"].tolist())

    for K in [1, 3, 5, 10]:
        metrics = compute_attack_metrics_target(queries_emb, pool_embs, pool_ids, target_ids, K=K)
        results.append({
            "ratio": ratio_name,
            "k": K,
            "num_queries": queries_emb.shape[0],
            "num_target_docs": len(target_ids),
            "num_pool_docs": len(pool_ids),
            **metrics
        })

# ====================== LƯU KẾT QUẢ ======================
results_df = pd.DataFrame(results)
out_path = HOTFLIP_DIR / "result" / "attack_metrics_full_targetted.csv"
results_df.to_csv(out_path, index=False)

print(f"\n✅ Saved metrics to {out_path}")
print(results_df.head(10))

In [15]:
import pandas as pd
from pathlib import Path

# --- Thư mục gốc ---
HOTFLIP_DIR = Path("/Users/hieunguyen/Downloads/SANNER_2025/data/hotflip_result/result")
RESULT_DIR = HOTFLIP_DIR 

# --- File poison và target ---
poison_files = [
    "attack_metrics_full.csv",
    "avg_poison_in_top10.csv",
    "custom_p_at_k_poison.csv"
]

target_files = [
    "attack_metrics_full_targetted.csv",
    "avg_targeted_in_top10.csv",
    "custom_p_at_k_targetted.csv"
]

def merge_files(files_list, suffix):
    merged = None
    for fname in files_list:
        fpath = HOTFLIP_DIR / fname
        if not fpath.exists():
            print(f"⚠️ File not found: {fpath}")
            continue
        df = pd.read_csv(fpath)

        # Nếu file không có cột k, thêm k=10
        if "k" not in df.columns:
            df["k"] = 10

        # Đổi tên các cột metric trừ ratio, k
        rename_dict = {col: f"{col}_{suffix}" for col in df.columns if col not in ["ratio","k"]}
        df = df.rename(columns=rename_dict)

        # Merge
        if merged is None:
            merged = df
        else:
            merged = pd.merge(merged, df, on=["ratio","k"], how="outer")
    return merged

# --- Merge poison và target ---
merged_poison = merge_files(poison_files, "poison")
merged_target = merge_files(target_files, "target")

# --- Merge poison + target theo ratio + k ---
merged_all = pd.merge(merged_poison, merged_target, on=["ratio","k"], how="outer")

# --- Sắp xếp cột theo cặp poison/target ---
fixed_cols = ["ratio","k"]
all_cols = [c for c in merged_all.columns if c not in fixed_cols]

# Lấy tên metric cơ bản, bỏ hậu tố
metrics = sorted(set([c.rsplit("_",1)[0] for c in all_cols]))

ordered_cols = fixed_cols[:]
for m in metrics:
    poison_col = f"{m}_poison"
    target_col = f"{m}_target"
    if poison_col in merged_all.columns:
        ordered_cols.append(poison_col)
    if target_col in merged_all.columns:
        ordered_cols.append(target_col)

merged_all = merged_all[ordered_cols]

# --- Lưu kết quả ---
out_path = RESULT_DIR / "result.csv"
merged_all.to_csv(out_path, index=False)
print(f"✅ Saved merged & paired CSV to {out_path}")
print(merged_all.head(10))



✅ Saved merged & paired CSV to /Users/hieunguyen/Downloads/SANNER_2025/data/hotflip_result/result/result.csv
               ratio   k  attack_success@k_poison  attack_success@k_target  \
0  ratio_0_1_percent   1                   0.0357                   0.0357   
1  ratio_0_1_percent   3                   0.1001                   0.1001   
2  ratio_0_1_percent   5                   0.1618                   0.1618   
3  ratio_0_1_percent  10                   0.2906                   0.2906   
4  ratio_0_5_percent   1                   0.0357                   0.0357   
5  ratio_0_5_percent   3                   0.1001                   0.1001   
6  ratio_0_5_percent   5                   0.1618                   0.1618   
7  ratio_0_5_percent  10                   0.2906                   0.2906   
8  ratio_1_0_percent   1                   0.0357                   0.0357   
9  ratio_1_0_percent   3                   0.1001                   0.1001   

   avg_poison_in_top10_poison  a